# DataFrame/DataSet编程指南

## 起点：SparkSession

SparkSession类是Spark中所有功能的入口点。要创建一个基本的SparkSession，只需使用SparkSession.builder：

```python
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
```

SparkSessionSpark 2.0中的内置支持Hive功能，包括使用HiveQL编写查询，访问Hive UDF以及从Hive表读取数据的功能。要使用这些功能，您不需要现有的Hive设置。

## 创建DataFrame

```python
# spark is an existing SparkSession
df = spark.read.json("examples/src/main/resources/people.json")
# Displays the content of the DataFrame to stdout
df.show()
# +----+-------+
# | age|   name|
# +----+-------+
# |null|Michael|
# |  30|   Andy|
# |  19| Justin|
# +----+-------+
```

## 无类型的DataSet操作

DataFrame为Scala，Java，Python和R中的结构化数据操作提供了一种特定于域的语言。

如上所述，在Spark 2.0中，DataFrames只是RowScala和Java API中的的数据集。与强类型的Scala / Java数据集附带的“类型转换”相反，这些操作也称为“非类型转换”。

这里我们提供一些使用数据集进行结构化数据处理的基本示例：

在Python中，可以通过属性（df.age）或索引（df['age']）访问DataFrame的列。尽管前者便于交互式数据探索，但强烈建议用户使用后者形式，后者是未来的证明，并且不会与列名保持一致，列名也是DataFrame类的属性。

```python
# spark, df are from the previous example
# Print the schema in a tree format
df.printSchema()
# root
# |-- age: long (nullable = true)
# |-- name: string (nullable = true)

# Select only the "name" column
df.select("name").show()
# +-------+
# |   name|
# +-------+
# |Michael|
# |   Andy|
# | Justin|
# +-------+

# Select everybody, but increment the age by 1
df.select(df['name'], df['age'] + 1).show()
# +-------+---------+
# |   name|(age + 1)|
# +-------+---------+
# |Michael|     null|
# |   Andy|       31|
# | Justin|       20|
# +-------+---------+

# Select people older than 21
df.filter(df['age'] > 21).show()
# +---+----+
# |age|name|
# +---+----+
# | 30|Andy|
# +---+----+

# Count people by age
df.groupBy("age").count().show()
# +----+-----+
# | age|count|
# +----+-----+
# |  19|    1|
# |null|    1|
# |  30|    1|
# +----+-----+
```

有关可在DataFrame上执行的操作类型的完整列表，请参阅API文档。

除了简单的列引用和表达式外，DataFrames还具有丰富的函数库，包括字符串处理，日期算术，通用数学运算等。完整列表可在DataFrame Function Reference中找到。

## 以编程方式运行SQL查询

SparkSession上的sql函数使应用程序能够以编程方式运行SQL查询，并以形式返回结果DataFrame。

```python
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()
# +----+-------+
# | age|   name|
# +----+-------+
# |null|Michael|
# |  30|   Andy|
# |  19| Justin|
# +----+-------+
```

## 全局临时视图

Spark SQL中的临时视图是会话作用域的，如果创建它的会话终止，它将消失。如果您希望拥有一个在所有会话之间共享的临时视图，并且在Spark应用程序终止之前一直保持活动状态，则可以创建全局临时视图。全局临时视图与系统保留的数据库相关联global_temp，我们必须使用限定名称来引用它，例如SELECT * FROM global_temp.view1。

```python
# Register the DataFrame as a global temporary view
df.createGlobalTempView("people")

# Global temporary view is tied to a system preserved database `global_temp`
spark.sql("SELECT * FROM global_temp.people").show()
# +----+-------+
# | age|   name|
# +----+-------+
# |null|Michael|
# |  30|   Andy|
# |  19| Justin|
# +----+-------+

# Global temporary view is cross-session
spark.newSession().sql("SELECT * FROM global_temp.people").show()
# +----+-------+
# | age|   name|
# +----+-------+
# |null|Michael|
# |  30|   Andy|
# |  19| Justin|
# +----+-------+
```

## 创建DataSet

数据集类似于RDD，但是，它们不使用Java序列化或Kryo，而是使用专用的Encoder对对象进行序列化以进行处理或通过网络传输。虽然编码器和标准序列化都负责将对象转换为字节，但是编码器是动态生成的代码，并使用一种格式，该格式允许Spark执行许多操作，如过滤，排序和哈希处理，而无需将字节反序列化为对象。

## 与RDD互操作

Spark SQL支持两种将现有RDD转换为数据集的方法。第一种方法使用反射来推断包含特定对象类型的RDD的架构。这种基于反射的方法可以使代码更简洁，并且当您在编写Spark应用程序时已经了解架构时，效果很好。

创建数据集的第二种方法是通过编程界面，该界面允许您构造模式，然后将其应用于现有的RDD。尽管此方法较为冗长，但可以在运行时才知道列及其类型的情况下构造数据集。


### 使用反射推断架构

Spark SQL可以将Row对象的RDD转换为DataFrame，从而推断数据类型。通过将键/值对列表作为kwargs传递给Row类来构造行。该列表的键定义表的列名，并且通过对整个数据集进行采样来推断类型，类似于对JSON文件执行的推断。

```python
from pyspark.sql import Row

sc = spark.sparkContext

# Load a text file and convert each line to a Row.
lines = sc.textFile("examples/src/main/resources/people.txt")
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

# Infer the schema, and register the DataFrame as a table.
schemaPeople = spark.createDataFrame(people)
schemaPeople.createOrReplaceTempView("people")

# SQL can be run over DataFrames that have been registered as a table.
teenagers = spark.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")

# The results of SQL queries are Dataframe objects.
# rdd returns the content as an :class:`pyspark.RDD` of :class:`Row`.
teenNames = teenagers.rdd.map(lambda p: "Name: " + p.name).collect()
for name in teenNames:
    print(name)
# Name: Justin
```

### 以编程方式指定架构

如果无法提前定义kwarg的字典（例如，记录的结构编码为字符串，或者将解析文本数据集，并且对不同用户使用不同的字段投影方式），则DataFrame可以通过编程方式创建三个脚步。

1. 从原始RDD创建元组或列表的RDD；
2. StructType在步骤1中创建的RDD中创建由元组或列表结构的匹配表示的模式。
3. 通过createDataFrame提供的方法将架构应用于RDD SparkSession。

例如：

```python
# Import data types
from pyspark.sql.types import *

sc = spark.sparkContext

# Load a text file and convert each line to a Row.
lines = sc.textFile("examples/src/main/resources/people.txt")
parts = lines.map(lambda l: l.split(","))
# Each line is converted to a tuple.
people = parts.map(lambda p: (p[0], p[1].strip()))

# The schema is encoded in a string.
schemaString = "name age"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

# Apply the schema to the RDD.
schemaPeople = spark.createDataFrame(people, schema)

# Creates a temporary view using the DataFrame
schemaPeople.createOrReplaceTempView("people")

# SQL can be run over DataFrames that have been registered as a table.
results = spark.sql("SELECT name FROM people")

results.show()
# +-------+
# |   name|
# +-------+
# |Michael|
# |   Andy|
# | Justin|
# +-------+
```



## 标量函数

标量函数是每行返回一个值的函数，聚合函数则返回一组行的值，而标量函数则是每行返回一个值。Spark SQL支持多种内置标量函数。它还支持用户定义的标量函数。

## 聚合函数

聚合函数是在一组行上返回单个值的函数。该内置聚合函数提供通用聚合如count()，countDistinct()，avg()，max()，min()，等用户不限于预定义的聚集功能，可以创建自己的。有关用户定义的聚合函数的更多详细信息，请参阅用户定义的聚合函数的文档 。